In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os

ROOT_DIR = os.path.abspath('..')
sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from neuron.neurons import UserTest, Neuron, User
import os

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\.venv\Lib\site-packages\accelerate\utils\other.py:220: DeprecationWarning: numpy.core is deprecated and has been renamed to numpy._core. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.multiarray.
  np.core.multiarray._reconstruct,


In [ ]:
llm_config = {
    "config_list": [
        {
            "client": "groq",
            "temperature": 0.0,
            "model": "llama3-70b-8192",
            "api_key": os.getenv("GROQ_API_KEY")
        }
    ]
}

from typing import Literal

Operator = Literal["+", "-", "*", "/"]

def exchange_rate(base_currency: str, quote_currency: str) -> float:
    """
    Returns the exchange rate between two currencies.

    Parameters:
    - base_currency (str): The currency you want to convert from (e.g., "USD").
    - quote_currency (str): The currency you want to convert to (e.g., "EUR").

    Returns:
    - float: The exchange rate from base_currency to quote_currency.

    Raises:
    - ValueError: If the currency pair is unknown or not supported.
    """

    # If the base and quote currencies are the same, return 1.0
    if base_currency == quote_currency:
        return 1.0

    # Conversion from USD to EUR (example rate)
    elif base_currency == "USD" and quote_currency == "EUR":
        return 1 / 1.09

    # Conversion from EUR to USD (example rate)
    elif base_currency == "EUR" and quote_currency == "USD":
        return 5.1

    # Raise error if the currency pair is not recognized
    else:
        raise ValueError(f"Unknown currencies {base_currency}, {quote_currency}")



def currency_calculator(
        base_amount: float,
        base_currency:str,
        quote_currency:str,
) -> str:
    """
    Converts a monetary amount from one currency to another.

    Parameters:
    - base_amount (float): The amount of money in the base currency.
    - base_currency (str): The currency in which the base_amount is expressed.
    - quote_currency (str): The target currency to convert the base_amount into.

    Returns:
    - str: A string representing the converted amount followed by the quote currency.
    """

    # Get the converted amount using the exchange rate
    quote_amount = exchange_rate(base_currency, quote_currency) * float(base_amount)

    # Return the converted amount as a formatted string
    return f"{quote_amount} {quote_currency}"


def tree_estimator(
        region: str = "Earth",
        unit: str = "trees"
) -> str:
    """
    Estimates the number of trees on a given region (default is Earth).

    Parameters:
    - region (str): The geographical area to estimate tree count (default is "Earth").
    - unit (str): The unit of measurement to return (default is "trees").

    Returns:
    - str: A string representing the estimated number of trees.
    """

    # Static estimate based on global scientific studies
    if region.lower() == "earth":
        estimated_trees = 3_000_000_000_000  # 3 trillion trees
    else:
        estimated_trees = 0  # Could be expanded with specific regions

    return f"{estimated_trees:,} {unit}"

# Let's first define the assistant agent that suggests tool calls.
user = User(
    name="user"
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
math = Neuron(
    name="math",
    llm_config=llm_config,
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Think step by step and use the calculator tool when needed." \
    "if the user request anything other than calculation, reply gently.'",
    tools = [currency_calculator, tree_estimator],
    reflect_on_tool_use=True,
    #tool_call_summary_format="Tool call: {result}",
)



In [4]:
chat_result = user.initiate_chat(math, message="quanto é 334 EUR em USD?")

[user ⟶ math]:
quanto é 334 EUR em USD?
reponse ChatCompletion(id='chatcmpl-1504b49f-4ca5-4831-9c96-ee1536231485', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_p6d4', function=Function(arguments='{"base_amount": 334, "base_currency": "EUR", "quote_currency": "USD"}', name='currency_calculator'), type='function')], reasoning="Okay, the user is asking how much 334 EUR is in USD. I need to help with that. I remember there's a currency calculator tool available. Let me check the function signature for currency_calculator. It requires base_amount, base_currency, and quote_currency. So, I'll set base_amount to 334, base_currency to EUR, and quote_currency to USD. That should do it. I'll format the tool call correctly with these parameters.\n"))], created=1745594091, model='deepseek-r1-distill-llama-70b', object='chat.completion', system_f

c:\Users\FS-Ma\OneDrive\Documents\projects\neuron\neuron\capabilities\clients\models\groq.py:317: UserWarning: Cost calculation not available for model deepseek-r1-distill-llama-70b
  warnings.warn(f"Cost calculation not available for model {model}", UserWarning)


reponse ChatCompletion(id='chatcmpl-0b5fb2a0-363d-4364-9843-6f9e9b400ddb', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="<think>\nOkay, the user is asking how much 334 Euros is in US Dollars. I remember that I can't directly access real-time exchange rates, so I need to let them know that. I should be polite and suggest they check a reliable source for the most accurate and up-to-date information. Maybe I can also mention some common places they can find this info, like a currency converter website or a financial news site. I want to make sure they feel helped even though I can't provide the exact figure myself.\n</think>\n\nPara saber quanto é 334 EUR em USD, você precisaria da taxa de câmbio atual entre o Euro e o Dólar Americano. Infelizmente, não tenho acesso em tempo real a taxas de câmbio. Você pode verificar isso em um site de conversão de moedas confiável ou em um serviço financeiro atualizado.", role='assistant', function_

In [5]:
user._oai_messages

defaultdict(list,
            {<neuron.neurons.neuron.Neuron at 0x24b85f93fb0>: [{'content': 'quanto é 334 EUR em USD?',
               'name': 'user',
               'role': 'assistant'},
              {'content': '[FunctionCall(id=\'call_p6d4\', arguments=\'{"base_amount": 334, "base_currency": "EUR", "quote_currency": "USD"}\', name=\'currency_calculator\')]',
               'name': 'math',
               'role': 'user'},
              {'content': "[FunctionExecutionResult(content='1703.3999999999999 USD', name='currency_calculator', call_id='call_p6d4', is_error=False)]",
               'name': 'math',
               'role': 'user'},
              {'content': "<think>\nOkay, the user is asking how much 334 Euros is in US Dollars. I remember that I can't directly access real-time exchange rates, so I need to let them know that. I should be polite and suggest they check a reliable source for the most accurate and up-to-date information. Maybe I can also mention some common places the

In [6]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': 'quanto é 334 EUR em USD?', 'name': 'user', 'role': 'assistant'}, {'content': '[FunctionCall(id=\'call_p6d4\', arguments=\'{"base_amount": 334, "base_currency": "EUR", "quote_currency": "USD"}\', name=\'currency_calculator\')]', 'name': 'math', 'role': 'user'}, {'content': "[FunctionExecutionResult(content='1703.3999999999999 USD', name='currency_calculator', call_id='call_p6d4', is_error=False)]", 'name': 'math', 'role': 'user'}, {'content': "<think>\nOkay, the user is asking how much 334 Euros is in US Dollars. I remember that I can't directly access real-time exchange rates, so I need to let them know that. I should be polite and suggest they check a reliable source for the most accurate and up-to-date information. Maybe I can also mention some common places they can find this info, like a currency converter website or a financial news site. I want to make sure they feel helped even though I can't provide the exact figure myself.\n<